### Import Libraries

In [ ]:
import numpy as np
import xarray as xr
from datetime import datetime, timedelta
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from mpl_toolkits.axes_grid1 import AxesGrid
from IPython.display import Image, display

### Load netcdf into xarray dataset object. 

In [ ]:
# filename with location
filename = '../sample-data/erai.mon.t2m.2000-2005.nc'

ds = xr.open_dataset(filename)
print('ds size in GB {:0.2f}\n'.format(ds.nbytes / 1e9))
ds.info()

ds.info() tells us about our dataset. It has 241 latitude points (90 to -90; 0.25 degree resolution), 480 longitude points (0-360; 0.25 degree resolution), 72 time points (monthly from Jan 2000 to December 2005), and it has a variable 't2m' which is 2 metre temperature. 

### Convert temperature to Celsius

In [ ]:
ds['t2m'] = ds.t2m - 273.15
ds['t2m'] = ds.t2m.assign_attrs(units='C')
ds

### Subset dataset to the US

In [ ]:
subset_ds = ds.sel(latitude=slice(55,10), longitude=slice(360-135,360-60))
subset_ds

### Plot first timestep using xarray plotting function and an orthographic projection.

In [ ]:
ax = plt.axes(projection=ccrs.Orthographic(-80, 35))

subset_ds.t2m.isel(time=0).plot.contourf(ax=ax, transform=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()

### Plot the first 9 time steps in a panel plot with a single colorbar using xarray's plot function.

In [ ]:
g_simple = subset_ds.t2m[:9].plot(x='longitude', y='latitude', col='time', col_wrap=3)

### Plot the first 9 time steps in a panel plot with a single colorbar using cartopy and save the figure.

In [ ]:
fig = plt.figure()
fig.dpi = 600
fname = 't2m_panel_plot'
fmt = 'png'
results = subset_ds.t2m[:9]
lats = subset_ds['latitude']
lons = subset_ds['longitude']

## currently this only works if you set the clevs manually
clevs = np.arange(-32,36,4)

for i in np.arange(len(results)):
    ax = plt.subplot(3, 3, i+1, projection=ccrs.PlateCarree())
    ax.set_extent([lons.min(), lons.max(), lats.min(), lats.max()], crs=ccrs.PlateCarree())
    cf = ax.contourf(lons, lats, results[i], transform=ccrs.PlateCarree(),
                     extend='neither', cmap='bwr', levels=clevs)
    
    ax.coastlines(linewidths=1.0)
    ax.set_title(pd.Timestamp(subset_ds.time[i].values), fontsize=10)

    ## Add in meridian and parallels
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=.5, color='black', alpha=0.5, linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xlocator = mticker.FixedLocator(np.flip(np.arange(360-lons.max()-20, 360-lons.min()+20, 20)*-1))
    gl.ylocator = mticker.FixedLocator(np.arange(lats.min(), lats.max()+10, 10))
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 7}
    gl.ylabel_style = {'size': 7}

# # add colorbar [left, bottom, width, height]
ax2 = fig.add_axes([1.0, 0.13, 0.03, 0.75])
cbar = fig.colorbar(cf, cax=ax2, drawedges=True, 
                    orientation='vertical', extendfrac='auto')
cbar.ax.tick_params(labelsize=8)
cbar.set_label(results.units, fontsize=8)

plt.subplots_adjust(hspace=0.5, wspace=0.3)

fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=fig.dpi)
fig.clf()


plotFile = fname + '.png'
print(plotFile)
display(Image(plotFile))